# Depression Detection Model for Twitter Accounts
Santiago Paiz

Ekaterina Podruzhko

Abdullah Al Saidi

Hisham Al Hashmi



# Discovery

In [2]:
import pandas as pd
import os

In [13]:
DATA_DEPRESSION_PATH = './data/depression/'

In [29]:
depression_dfs = {
    f.strip('.csv').replace(' ', "_"): pd.read_csv(DATA_DEPRESSION_PATH + f) for f in os.listdir(DATA_DEPRESSION_PATH)
    }

In [60]:
depression_dfs['I_am_fighting_depression']

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,depr_level
0,1434912344874766336,1434912342228176897,2021-09-06 16:12:17 UTC,2021-09-06,16:12:17,0,346387825,vizio32,Daddy Diddlez,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_am_fighting_depression
1,1434845574801641473,1434845574801641473,2021-09-06 11:46:57 UTC,2021-09-06,11:46:57,0,1023981894,tumisangmo,Champagne Mami,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_am_fighting_depression
2,1434707385159802882,1434678401986711554,2021-09-06 02:37:50 UTC,2021-09-06,02:37:50,0,1427264476047319055,metal_fuzz,MetalFuzz,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'DollBrat1985', 'name': '🎀 Do...",NaN,NaN,NaN,NaN,I_am_fighting_depression
3,1434652964438921231,1434652964438921231,2021-09-05 23:01:35 UTC,2021-09-05,23:01:35,0,4836449714,boughscat,whiskers,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_am_fighting_depression
4,1434600535076769796,1434600535076769796,2021-09-05 19:33:15 UTC,2021-09-05,19:33:15,0,543177827,original_lexi,Lexi,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_am_fighting_depression
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1428560192778682372,1428560190643781637,2021-08-20 03:31:05 UTC,2021-08-20,03:31:05,0,1422908475152142344,slaughts3,Slaughts,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_am_fighting_depression
76,1428552156773339139,1428551151339909127,2021-08-20 02:59:10 UTC,2021-08-20,02:59:10,0,1422762576723824642,pantherlewd,LewdPanther,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_am_fighting_depression
77,1428544067634438145,1428543062494515202,2021-08-20 02:27:01 UTC,2021-08-20,02:27:01,0,93528653,phattgreg,Greg Sully,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'TheTweetOfGod', 'name': 'God...",NaN,NaN,NaN,NaN,I_am_fighting_depression
78,1428342339106377732,1428335809464258565,2021-08-19 13:05:25 UTC,2021-08-19,13:05:25,0,1131609294540091392,hbanchey,HancheyBanchey,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'ScrotieMcB', 'name': 'Scrapp...",NaN,NaN,NaN,NaN,I_am_fighting_depression


In [59]:
depression_dfs['users_timeline']

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,depr_level
0,1435429475567607814,1435424118191575045,2021-09-08 02:27:10 UTC,2021-09-08,02:27:10,0,1391103833695158272,starkorla,orla_stark,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'msworldwide69', 'name': 'sar...",NaN,NaN,NaN,NaN,users_timeline
1,1435429339676299264,1435424118191575045,2021-09-08 02:26:38 UTC,2021-09-08,02:26:38,0,1391103833695158272,starkorla,orla_stark,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'msworldwide69', 'name': 'sar...",NaN,NaN,NaN,NaN,users_timeline
2,1435028936258248706,1434823227180822535,2021-09-06 23:55:34 UTC,2021-09-06,23:55:34,0,1391103833695158272,starkorla,orla_stark,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'ilyhariah', 'name': 'FaZe ha...",NaN,NaN,NaN,NaN,users_timeline
3,1435028173553504257,1435021286921621505,2021-09-06 23:52:32 UTC,2021-09-06,23:52:32,0,1391103833695158272,starkorla,orla_stark,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'Quackity', 'name': 'Quackity...",NaN,NaN,NaN,NaN,users_timeline
4,1434766350342189056,1434527317741686785,2021-09-06 06:32:09 UTC,2021-09-06,06:32:09,0,1391103833695158272,starkorla,orla_stark,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'Ignoredream', 'name': 'apple...",NaN,NaN,NaN,NaN,users_timeline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,1432460534385831937,1432455495990738947,2021-08-30 21:49:39 UTC,2021-08-30,21:49:39,0,920433840450977792,dotcombaby947,🪶𝖒𝖞𝖘. 𝖘𝖆𝖙𝖘𝖚𝖐𝖎 🪶,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
48891,1432460055203356672,1432455495990738947,2021-08-30 21:47:45 UTC,2021-08-30,21:47:45,0,920433840450977792,dotcombaby947,🪶𝖒𝖞𝖘. 𝖘𝖆𝖙𝖘𝖚𝖐𝖎 🪶,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
48892,1432456579631042564,1432455495990738947,2021-08-30 21:33:56 UTC,2021-08-30,21:33:56,0,920433840450977792,dotcombaby947,🪶𝖒𝖞𝖘. 𝖘𝖆𝖙𝖘𝖚𝖐𝖎 🪶,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
48893,1432456013026775041,1432455495990738947,2021-08-30 21:31:41 UTC,2021-08-30,21:31:41,0,920433840450977792,dotcombaby947,🪶𝖒𝖞𝖘. 𝖘𝖆𝖙𝖘𝖚𝖐𝖎 🪶,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline


In [44]:
for key, value in depression_dfs.items():
    print(f'TITLE: {key}')
    print(value.columns)
    print()


TITLE: I_suffer_from_depression
Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

TITLE: users_timeline
Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       't

In [37]:
import collections

In [45]:
if(collections.Counter(depression_dfs['I_suffer_from_depression'])==collections.Counter(depression_dfs['I_am_fighting_depression']) \
    and collections.Counter(depression_dfs['I_am_fighting_depression']) == collections.Counter(depression_dfs['I_am_diagnosed_with_depression'])
    and collections.Counter(depression_dfs['I_am_fighting_depression']) == collections.Counter(depression_dfs['users_timeline'])):
    print("Equal")
else:
    print("Non equal")


Equal


There are 4 different data tables, however all of them have the same schema, in order to have them all in a single data frame we join them together adding a new column which represented in which data it was before the merger.

In order to make it easier we add one column to each of the data frame with the names of which the users is. 

In [46]:
for key, value in depression_dfs.items():
    value['depr_level'] = key


In [51]:
depression_dfs['I_suffer_from_depression'].sample(2)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,depr_level
443,1432658443429502977,1432658443429502977,2021-08-31 10:56:05 UTC,2021-08-31,10:56:05,0,1417122361745154049,crldwnbgt_,Dawn,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,I_suffer_from_depression
343,1433208710982033411,1433117975310123019,2021-09-01 23:22:39 UTC,2021-09-01,23:22:39,0,2789574570,parsleyjane8,Jane Parsley 🆘,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'clairermassey', 'name': '𝓒𝓵𝓪...",NaN,NaN,NaN,NaN,I_suffer_from_depression


In [55]:
# here is the concat of every df
derp_data = pd.concat(list(depression_dfs.values()))

In [58]:
derp_data.sample(5)

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,depr_level
4961,1431668815885131780,1431668815885131780,2021-08-28 17:23:39 UTC,2021-08-28,17:23:39,0,1426671062356533248,matthasadhd,matt,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
35192,1433478420881498112,1433478420881498112,2021-09-02 17:14:22 UTC,2021-09-02,17:14:22,0,1390740031845789697,danieltuesta7,🇨🇺Daniel 🇺🇸Tuesta 🇵🇪,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline
23773,1434167837141131268,1433892527262863370,2021-09-04 14:53:52 UTC,2021-09-04,14:53:52,0,3084322089,papa49tw,Papa Steve,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'LoveMyKiaraM', 'name': 'Raye...",NaN,NaN,NaN,NaN,users_timeline
42425,1433143287150485513,1433120010633887748,2021-09-01 19:02:40 UTC,2021-09-01,19:02:40,0,181659599,beacher61,Kathleen Morgan,NaN,...,NaN,NaN,NaN,NaN,"[{'screen_name': 'MarieanneDH', 'name': 'RedBe...",NaN,NaN,NaN,NaN,users_timeline
7856,1435457240194904064,1435457240194904064,2021-09-08 04:17:30 UTC,2021-09-08,04:17:30,0,1174882312082714626,_deshanna_,-Nex/Deshanna-¹⁴³,NaN,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,users_timeline


# Data Analysis

# Something